# netCDF and xarray

Lets start by importing all packages, that we need.

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

## Let's start with an example 

Assume we have a weather forecast. Our data contains forcasts of temperature and precipitation for three days at four locations. That means we have to store the data as well as the locations latiude and longitude and the time for the forecast. Since we could have multiple forecasts we also want to store the reference_time from when the forecast was made. Using pandas we can store this data in a dataframe.

In [2]:
df = pd.DataFrame()
df['temperature'] = 15 + 8 * np.random.randn(12)
df['precipitation'] = 10 * np.random.rand(12)
df['lon'] = np.array([-99.83, -99.83, -99.83, -99.32, -99.32, -99.32, -99.79, -99.79, -99.79, -99.23, -99.23, -99.23])
df['lat'] = np.array([42.25, 42.25, 42.25, 42.21, 42.21, 42.21, 42.63, 42.63, 42.63, 42.59, 42.59, 42.59])
df['time'] = np.array([pd.date_range('2014-09-06', periods=3)] * 4).reshape(12)
df['reference_time'] = np.array([pd.Timestamp('2014-09-05')] * 12)
df

,temperature,precipitation,lon,lat,time,reference_time
0,9.886983,5.397835,-99.83,42.25,2014-09-06,2014-09-05
1,18.736250,0.319311,-99.83,42.25,2014-09-07,2014-09-05
2,26.933078,7.437245,-99.83,42.25,2014-09-08,2014-09-05
3,18.902163,3.957231,-99.32,42.21,2014-09-06,2014-09-05
4,6.071471,4.507137,-99.32,42.21,2014-09-07,2014-09-05
5,17.095652,4.904113,-99.32,42.21,2014-09-08,2014-09-05
6,25.689399,0.434806,-99.79,42.63,2014-09-06,2014-09-05
7,13.847594,3.784362,-99.79,42.63,2014-09-07,2014-09-05
8,21.624169,7.718258,-99.79,42.63,2014-09-08,2014-09-05
9,16.628470,8.883603,-99.23,42.59,2014-09-06,2014-09-05


In this dataframe we have a generic index and six columns. We could also store the data with just two columns for temperature and precipitation and have a four column multiindex. Maybe we want to slice the data or apply functions along the time or space axis. Then it would be helpful if we hadn't stored the data tabular but in a N-dimensional array. This is a case where we can use netCDF!

## What is netCDF

NetCDF is a self-describing, machine-independent data format. Self-describing means, that all information regarding the data, eg. metadata, labels, units are also stored in the file. It is developed by Unidata and there are software libraries in many languages to interact with netCDF files.

The commen data format (CDF) was originally developed by Nasa and netCDF is build upon it. Data is stored in variables, dimensions and attributes.
Variables are N-dimensional arrays that store the actual data.
Dimensions are used to label the N-dim arrays.
Attributes are used to store additional information about the data.

NetCDF-4 introduced the enhanced data model. 
It allows you to store groups of data within one file and added HDF5 support for netCDF.
Before netCDF-4 you could only have one unlimited dimension, which is a dimension where you don't have to specify the size. Since netCDF-4 you can have multiple unlimited dimensions.

Our example from before can now be stored like this:

<img src="http://xarray.pydata.org/en/stable/_images/dataset-diagram.png", width=800, height=60>

In [3]:
temp = 15 + 8 * np.random.randn(2, 2, 3)

precip = 10 * np.random.rand(2, 2, 3)

lon = [[-99.83, -99.32], [-99.79, -99.23]]

lat = [[42.25, 42.21], [42.63, 42.59]]

ds = xr.Dataset({'temperature': (['x', 'y', 'time'],  temp),
                 'precipitation': (['x', 'y', 'time'], precip)},
                 coords={'lon': (['x', 'y'], lon),
                         'lat': (['x', 'y'], lat),
                         'time': pd.date_range('2014-09-06', periods=3),
                         'reference_time': pd.Timestamp('2014-09-05')},
                 attrs={'units': 'Celsius, l/m²'})
ds

<xarray.Dataset>
Dimensions:         (time: 3, x: 2, y: 2)
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y
Data variables:
    temperature     (x, y, time) float64 20.15 14.46 3.847 16.26 9.526 22.48 ...
    precipitation   (x, y, time) float64 5.0 2.887 7.085 2.596 8.827 2.864 ...
Attributes:
    units:    Celsius, l/m²

NetCDF is a file format to store data in. In order to work with netCDF we need software that can interact with it. For our example we used xarray.

## What is xarray

Xarray is an open source python package that aspires to be the 'pandas for N-dim arrays'. There are many similarities between xarray and pandas or numpy and there are apis that allow you to convert data from one to the other. Xarray uses the netCDF data model, so you can think of xarray as an 'in-memory' equivalent of netCDF. That makes it a perfect choice for working with netCDF. 

Xarray consists of two data structures, data arrays and data sets.

### DataArray

Data arrays are multidimensional labeled arrays. A data array consists of:

* values: a numpy.ndarray holding the array’s values
* dims: dimension names for each axis (e.g., ('x', 'y', 'z'))
* coords: a dict-like container of arrays (coordinates) that label each point 
* attrs: an OrderedDict to hold arbitrary metadata (attributes)

Here is an example of a data array that holds 96 labeled values over one day at three german cities:


In [4]:
data = np.random.rand(96, 3)

locs = ['Freiburg', 'Oldenburg', 'Karlsruhe']

times = pd.date_range('2000-01-01', periods=96, freq='15Min')

data = xr.DataArray(data, coords=[times, locs], dims=['time', 'space'])
data

<xarray.DataArray (time: 96, space: 3)>
array([[ 0.548175,  0.614043,  0.964418],
       [ 0.376261,  0.224052,  0.215249],
       [ 0.148024,  0.145431,  0.37637 ],
       ..., 
       [ 0.274927,  0.885318,  0.443139],
       [ 0.262893,  0.850894,  0.062152],
       [ 0.223332,  0.142055,  0.377829]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-01T00:15:00 ...
  * space    (space) <U9 'Freiburg' 'Oldenburg' 'Karlsruhe'

### Dataset

If you think of data arrays as N-dim pandas series, than a xarray dataset is the equivalent of a pandas dataframe in that it stores several data arrays as variables that share the same dimensions.

There are several ways of accesing the data similar to pandas and numpy:

In [ ]:
# numpy style
ds.temperature[1, 1, :]

# pandas style
ds.temperature.loc[:, :, '2014-09-07']

# xarray style
ds.sel(x = 1, y = 1, time='2014-09-07')

One more thing I want to mention about datasets is xarrays handling of datasets that are too large to fit in your memory.

### Datasets that are too large to fit in memory

Multidimensional data often comes in datasets that are too large to fit into memory. That can mean several gigabyte as well as terabytes of data. Xarray has been extended to use dask for parallel processing and out of memory computation. Dask divides arrays into many small pieces, called chunks, each of which is presumed to be small enough to fit into memory. Most of xarrays operations can be applied on dask arrays, which makes it easy to work with it. There are multiple ways of using xarray with dask:

In [ ]:
# open dataset while specify dask chunks.
ds = xr.open_dataset('example-data.nc', chunks={'time': 10})

# opens multiple netCDF files at once, each file is per default chunked into a dask array.
xr.open_mfdataset('my/files/*.nc')